 # Analyze MFC used cars

## Problem Statement


## Objective


## Business Benefits



## Dataset Descriptions



# Data Loading and Analysis

## Loading the libraries and initialization of basic variables

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# data file paths
JTD_DATA_FILE = "../data/JTD.csv"
CUST_DATA_FILE = "../data/Customer_Data.xlsx"
PLANT_MASTER_FILE = "../data/Latest_Plant Master.xlsx"

# log level
LOG_LEVEL = 'INFO'

## Data Loading

In [ ]:
# load the data files for JTD, Customer and Plant data
jtd_df = pd.read_csv(JTD_DATA_FILE) 
cust_df = pd.read_excel(CUST_DATA_FILE)
plant_df = pd.read_excel(PLANT_MASTER_FILE)

FileNotFoundError: [Errno 2] No such file or directory: '../data/Latest_Plant Master.xlsx'

In [ ]:
def printBasicInfo(data, briefDesc):
    if LOG_LEVEL == 'INFO':
        print('Dataframe Brief Description for:', briefDesc)
        print('======='*12)
        print('\nDataframe Field Types')
        print(data.info())
        print('\nDataframe Field Null and NonNull fields')
        print(data.isnull().sum())
        print('\nDataframe head')
        print(data.head(5))

In [ ]:
printBasicInfo(jtd_df, briefDesc='JTD Dataset information')

In [ ]:
printBasicInfo(cust_df, briefDesc='Customer Dataset information')

In [ ]:
printBasicInfo(plant_df, briefDesc='Plant Dataset information')

## Data Description

